

https://neurohive.io/ru/tutorial/cnn-na-pytorch/


In [ ]:
num_epochs = 2
num_classes = 10
batch_size = 3
learning_rate = 0.001
DATA_PATH = '/mnist_train.csv'
# MODEL_STORE_PATH = '/home/denis/Документы/ml'

In [ ]:
import torch
from torchvision import datasets
from torchvision import transforms

Нейронная сеть обучается лучше, когда входные данные нормализованы так, что их значения находятся в диапазоне от -1 до 1 или от 0 до 1. Чтобы это сделать с помощью нормализации PyTorch, необходимо указать среднее и стандартное отклонение MNIST датасета, которые в этом случае равны 0.1307 и 0.3081 соответственно. Отметим, среднее значение и стандартное отклонение должны быть установлены для каждого входного канала. У MNIST есть только один канал, но уже для датасета CIFAR c 3 каналами (по одному на каждый цвет из RGB спектра) надо указывать среднее и стандартное отклонение для каждого.


In [ ]:
trans = transforms.Compose([transforms.ToTensor(), transforms.Normalize((0.1307,), (0.3081,))])

train_dataset = datasets.MNIST(root = 'data/', train = True, transform = trans, download=True)
test_dataset = datasets.MNIST(root='data/', train=False, transform=trans, download=True)

Failed to download (trying next):
HTTP Error 403: Forbidden



100%|██████████| 9912422/9912422 [00:00<00:00, 131683721.37it/s]


Extracting data/MNIST/raw/train-images-idx3-ubyte.gz to data/MNIST/raw

Failed to download (trying next):
HTTP Error 403: Forbidden



100%|██████████| 28881/28881 [00:00<00:00, 14736702.41it/s]

Extracting data/MNIST/raw/train-labels-idx1-ubyte.gz to data/MNIST/raw

Failed to download (trying next):
HTTP Error 403: Forbidden




100%|██████████| 1648877/1648877 [00:00<00:00, 58341766.95it/s]


Extracting data/MNIST/raw/t10k-images-idx3-ubyte.gz to data/MNIST/raw

Failed to download (trying next):
HTTP Error 403: Forbidden



100%|██████████| 4542/4542 [00:00<00:00, 6811057.84it/s]


Extracting data/MNIST/raw/t10k-labels-idx1-ubyte.gz to data/MNIST/raw



In [ ]:
from torch.utils.data import DataLoader

train_loader = DataLoader(dataset=train_dataset, batch_size=batch_size,shuffle=True)
test_loader = DataLoader(dataset=test_dataset, batch_size=batch_size, shuffle=False)

Это то место, где определяется модель. Наиболее простой способ создания структуры нейронной сети в PyTorch — создание наследственного класса от материнского nn.Module. Класс nn.Module очень полезен в PyTorch, он содержит всё необходимое для конструирования типичной глубокой нейронной сети. Кроме этого, класс имеет удобные функции: способы перемещения переменных и операций на GPU или обратно на CPU, применение рекурсивных функций через все свойства в классе (например, перенастройка всех весовых переменных), создание оптимизированных интерфейсов для тренировки и тому подобное. Все доступные методы можно посмотреть здесь.

In [ ]:
import torch.nn as nn
import torch.nn.functional as F

class ConvNet(nn.Module):
    def __init__(self, num_classes):
        super(ConvNet, self).__init__()

        self.fc1 = nn.Linear(784, 384)
        self.fc2 = nn.Linear(384, 128)
        self.fc3 = nn.Linear(128, num_classes)
        self.flatten = nn.Flatten()
        self.relu = nn.ReLU()

    def forward(self, x):
        # print("x before =", x.shape)
        # print("x size 0 =", x.size(0))
        # the size -1 is inferred from other dimensions
        x = x.reshape(x.size(0), -1)
        # print("x after =", x.shape)

        x = self.fc1(x)
        x = self.relu(x)
        x = self.fc2(x)
        x = self.relu(x)
        x = self.fc3(x)

        # print("x res =", x.shape)
        return x

In [ ]:
model = ConvNet(num_classes)
criterion = nn.CrossEntropyLoss()
optimizer = torch.optim.Adam(model.parameters(), lr=learning_rate)

https://blog.paperspace.com/writing-lenet5-from-scratch-in-python/

In [ ]:
model = ConvNet(num_classes)

#Setting the loss function
cost = nn.CrossEntropyLoss()

#Setting the optimizer with the model parameters and learning rate
optimizer = torch.optim.Adam(model.parameters(), lr=learning_rate)

#this is defined to print how many steps are remaining when training
total_step = len(train_loader)

In [ ]:
total_step = len(train_loader)
print(len(train_loader))
loss_list = []
acc_list = []
for epoch in range(num_epochs):
    for i, (images, labels) in enumerate(train_loader):
        # Прямой запуск
        outputs = model(images)
        loss = criterion(outputs, labels)
        loss_list.append(loss.item())

        # Обратное распространение и оптимизатор
        optimizer.zero_grad()
        loss.backward()
        optimizer.step()

        # Отслеживание точности
        total = labels.size(0)
        _, predicted = torch.max(outputs.data, 1)
        correct = (predicted == labels).sum().item()
        acc_list.append(correct / total)

        if (i + 1) % 100 == 0:
            print('Epoch [{}/{}], Step [{}/{}], Loss: {:.4f}, Accuracy: {:.2f}%'
                  .format(epoch + 1, num_epochs, i + 1, total_step, loss.item(),
                          (correct / total) * 100))
            for name, param in model.named_parameters():
              print(name, param)

Выходные данные были обрезаны до нескольких последних строк (5000).
        -9.0453e-03, -4.3764e-02, -5.0519e-02,  9.8043e-03, -5.0602e-02,
        -1.0592e-02,  1.4306e-02,  1.3307e-02, -3.8577e-02,  8.0734e-03,
        -1.5958e-05, -5.2830e-02, -2.0494e-03,  2.1553e-03, -5.2071e-02,
        -1.0920e-02, -2.5422e-02,  3.2353e-03,  1.9015e-02,  1.3941e-02,
        -5.2962e-03, -2.8769e-02,  2.1815e-02,  2.9414e-02, -3.1551e-02,
        -4.0010e-02,  2.8243e-02,  5.2265e-03, -1.2866e-02,  3.4614e-03,
         2.3652e-02, -3.9650e-02, -1.4248e-02,  5.0570e-03, -9.1379e-04,
        -2.8338e-02, -4.9714e-02,  9.8274e-03,  1.6411e-02, -7.0288e-04,
         2.1700e-02, -3.7820e-02, -1.0536e-02, -2.2264e-02, -2.9524e-02,
        -7.0974e-04,  8.0707e-03, -1.6424e-02,  1.9439e-02, -4.6523e-02,
         3.6936e-02, -5.3768e-02,  1.8574e-02,  2.4254e-03],
       requires_grad=True)
fc2.weight Parameter containing:
tensor([[-0.0200,  0.0002,  0.0352,  ..., -0.0410,  0.0031, -0.0171],
        [ 0

KeyboardInterrupt: 

In [ ]:
MODEL_STORE_PATH = '/home/denis/Документы/ml/'

model.eval()
with torch.no_grad():
    correct = 0
    total = 0
    for images, labels in test_loader:
        outputs = model(images)
        _, predicted = torch.max(outputs.data, 1)
        total += labels.size(0)
        correct += (predicted == labels).sum().item()

    print('Test Accuracy of the model on the 10000 test images: {} %'.format((correct / total) * 100))

# Сохраняем модель и строим график
# torch.save(model.state_dict(), MODEL_STORE_PATH + 'conv_net_model.ckpt')

Test Accuracy of the model on the 10000 test images: 95.83 %


3 полносвязных слоя - 89.8 %
3 ПС и 2 релу между ними - 95.83 %
